In [1]:
import pandas as pd
import csv

jeopardy = pd.read_csv("jeopardy.csv")

jeopardy.head()

,Show Number,Air Date,Round,Category,Value,Question,Answer
0,4680,2004-12-31,Jeopardy!,HISTORY,$200,"For the last 8 years of his life, Galileo was ...",Copernicus
1,4680,2004-12-31,Jeopardy!,ESPN's TOP 10 ALL-TIME ATHLETES,$200,No. 2: 1912 Olympian; football star at Carlisl...,Jim Thorpe
2,4680,2004-12-31,Jeopardy!,EVERYBODY TALKS ABOUT IT...,$200,The city of Yuma in this state has a record av...,Arizona
3,4680,2004-12-31,Jeopardy!,THE COMPANY LINE,$200,"In 1963, live on ""The Art Linkletter Show"", th...",McDonald's
4,4680,2004-12-31,Jeopardy!,EPITAPHS & TRIBUTES,$200,"Signer of the Dec. of Indep., framer of the Co...",John Adams


In [2]:
jeopardy.columns

Index(['Show Number', ' Air Date', ' Round', ' Category', ' Value',
       ' Question', ' Answer'],
      dtype='object')

In [3]:
import re

def normalize_text(text):
    text = text.lower()
    text = re.sub("[^A-Za-z0-9\s]","",text)
    text = re.sub("\s+","",text)
    return text
def normalize_values(text):
    text = re.sub(r'\$',"",str(text))
    text = re.sub("[^A-Za-z0-9\s]","",text)
    text = float(text)
    try:
        text = int(text)
    except Exception:
        text = 0
    return text

In [4]:
jeopardy.columns = ['Show Number','Air Date','Round','Category','Value','Question','Answer']

In [5]:
jeopardy["clean_question"] = jeopardy["Question"].apply(normalize_text)
jeopardy["clean_answer"] = jeopardy["Answer"].apply(normalize_text)
jeopardy["clean_value"] = jeopardy["Value"].apply(normalize_values)

In [6]:
jeopardy.head(6)

,Show Number,Air Date,Round,Category,Value,Question,Answer,clean_question,clean_answer,clean_value
0,4680,2004-12-31,Jeopardy!,HISTORY,$200,"For the last 8 years of his life, Galileo was ...",Copernicus,forthelast8yearsofhislifegalileowasunderhousea...,copernicus,200
1,4680,2004-12-31,Jeopardy!,ESPN's TOP 10 ALL-TIME ATHLETES,$200,No. 2: 1912 Olympian; football star at Carlisl...,Jim Thorpe,no21912olympianfootballstaratcarlisleindiansch...,jimthorpe,200
2,4680,2004-12-31,Jeopardy!,EVERYBODY TALKS ABOUT IT...,$200,The city of Yuma in this state has a record av...,Arizona,thecityofyumainthisstatehasarecordaverageof405...,arizona,200
3,4680,2004-12-31,Jeopardy!,THE COMPANY LINE,$200,"In 1963, live on ""The Art Linkletter Show"", th...",McDonald's,in1963liveontheartlinklettershowthiscompanyser...,mcdonalds,200
4,4680,2004-12-31,Jeopardy!,EPITAPHS & TRIBUTES,$200,"Signer of the Dec. of Indep., framer of the Co...",John Adams,signerofthedecofindepframeroftheconstitutionof...,johnadams,200
5,4680,2004-12-31,Jeopardy!,3-LETTER WORDS,$200,"In the title of an Aesop fable, this insect sh...",the ant,inthetitleofanaesopfablethisinsectsharedbillin...,theant,200


In [7]:
jeopardy["Air Date"] = pd.to_datetime(jeopardy["Air Date"])

In [8]:
jeopardy.dtypes

Show Number                int64
Air Date          datetime64[ns]
Round                     object
Category                  object
Value                     object
Question                  object
Answer                    object
clean_question            object
clean_answer              object
clean_value                int64
dtype: object

In [9]:
def count_matches(row):
    split_answer = row["clean_answer"].split()
    split_question = row["clean_question"].split()
    if "the" in split_answer:
        split_answer.remove("the")
    if len(split_answer) == 0:
        return 0
    match_count = 0
    for item in split_answer:
        if item in split_question:
            match_count += 1
    return match_count / len(split_answer)

jeopardy["answer_in_question"] = jeopardy.apply(count_matches,axis =1)

In [10]:
jeopardy["answer_in_question"].value_counts()

answer_in_question
0.0    19999
Name: count, dtype: int64

In [11]:
question_overlap = []
terms_used = set()

jeopardy = jeopardy.sort_values("Air Date")

for i, row in jeopardy.iterrows():
        split_question = row["clean_question"].split(" ")
        split_question = [q for q in split_question if len(q) > 5]
        match_count = 0
        for word in split_question:
            if word in terms_used:
                match_count += 1
        for word in split_question:
            terms_used.add(word)
        if len(split_question) > 0:
            match_count /= len(split_question)
        question_overlap.append(match_count)
jeopardy["question_overlap"] = question_overlap

jeopardy["question_overlap"].mean()

np.float64(0.000600030001500075)

In [12]:
jeopardy["Value"].value_counts()

Value
$400      3892
$800      2980
$200      2784
$600      1890
$1000     1796
          ... 
$7,400       1
$2,127       1
$367         1
$5,800       1
$8,200       1
Name: count, Length: 75, dtype: int64

In [13]:
def determine_value(row):
    value = 0
    if row["clean_value"] > 800:
        value = 1
    return value

jeopardy["high_value"] = jeopardy.apply(determine_value,axis =1)

In [14]:
jeopardy["high_value"].value_counts()

high_value
0    14265
1     5734
Name: count, dtype: int64

In [15]:
def count_usage(term):
    low_count = 0
    high_count = 0
    for i, row in jeopardy.iterrows():
        if term in row["clean_question"].split(" "):
            if row["high_value"] == 1:
                high_count += 1
            else:
                low_count += 1
    return high_count, low_count


In [16]:
from random import choice

terms_used_list = list(terms_used)
comparison_terms = [choice(terms_used_list) for _ in range(10) ]

observed_expected = []

for term in comparison_terms:
    observed_expected.append(count_usage(term))
    
observed_expected

[(1, 0),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (0, 1),
 (1, 0)]

In [21]:
from scipy.stats import chisquare
import numpy as np

high_value_count = jeopardy[jeopardy["high_value"] == 1].shape[0]
low_value_count = jeopardy[jeopardy["high_value"] == 0].shape[0]

chi_squared = []

for obs in observed_expected:
    total = sum(obs)
    total_prop = total / jeopardy.shape[0]
    high_value_exp = total_prop * high_value_count
    low_value_exp = total_prop * low_value_count
    
    observed = np.array([obs[0],obs[1]])
    expected = np.array([high_value_exp,low_value_exp])
    chi_squared.append(chisquare(observed,expected))
    
chi_squared

[Power_divergenceResult(statistic=np.float64(2.487792117195675), pvalue=np.float64(0.11473257634454047)),
 Power_divergenceResult(statistic=np.float64(0.401962846126884), pvalue=np.float64(0.5260772985705469)),
 Power_divergenceResult(statistic=np.float64(0.401962846126884), pvalue=np.float64(0.5260772985705469)),
 Power_divergenceResult(statistic=np.float64(0.401962846126884), pvalue=np.float64(0.5260772985705469)),
 Power_divergenceResult(statistic=np.float64(0.401962846126884), pvalue=np.float64(0.5260772985705469)),
 Power_divergenceResult(statistic=np.float64(0.401962846126884), pvalue=np.float64(0.5260772985705469)),
 Power_divergenceResult(statistic=np.float64(0.401962846126884), pvalue=np.float64(0.5260772985705469)),
 Power_divergenceResult(statistic=np.float64(0.401962846126884), pvalue=np.float64(0.5260772985705469)),
 Power_divergenceResult(statistic=np.float64(0.401962846126884), pvalue=np.float64(0.5260772985705469)),
 Power_divergenceResult(statistic=np.float64(2.4877921